<b><h2>  Mitigating the bias

In [1]:
import pandas as pd
import numpy as np

In [2]:
random_state = 100

In [3]:
X_train = pd.read_csv("X_train.csv", index_col=0)
X_test = pd.read_csv("X_test.csv",index_col=0)
y_train = pd.read_csv("y_train.csv",index_col=0)
y_test = pd.read_csv("y_test.csv",index_col=0)

baseline_preprocessed_df = pd.read_csv("data_preprocessed_baseline.csv",index_col=0)
baseline_predictions_df = pd.read_csv("predictions_logistic.csv",index_col=0)

In [4]:
X_train.head()

,id,sex,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,c_charge_desc,time_in_custody_in_days,time_in_jail_in_days,...,Native American_race,Other_race,Felony_c_charge_degree,Misdemeanor_c_charge_degree,High_score_text,Low_score_text,Medium_score_text,High_v_score_text,Low_v_score_text,Medium_v_score_text
5375,8193,1,20,0,0,0,0,0.592456,61.0,1.0,...,0,0,1,0,0,0,1,0,0,1
3605,5507,1,43,0,0,0,2,0.383073,0.0,0.0,...,0,1,0,1,0,1,0,0,1,0
29,39,1,26,0,0,1,5,0.497714,8.0,0.0,...,0,0,1,0,0,0,1,0,0,1
2420,3692,1,41,0,0,0,8,0.534040,0.0,1.0,...,0,0,1,0,0,1,0,0,1,0
5641,8602,1,24,0,0,0,5,0.441770,39.0,37.0,...,0,0,1,0,0,0,1,0,1,0


<h3> Evaluate baseline model predictions once again

In [5]:
from sklearn.metrics import classification_report

In [6]:
baseline_perf = pd.DataFrame(classification_report(y_true=baseline_predictions_df['y_true_labels'], y_pred=baseline_predictions_df['y_pred_labels'],output_dict=True)).T
baseline_perf

,precision,recall,f1-score,support
0,0.722861,0.716456,0.719644,790.000000
1,0.657492,0.664606,0.661030,647.000000
accuracy,0.693111,0.693111,0.693111,0.693111
macro avg,0.690177,0.690531,0.690337,1437.000000
weighted avg,0.693429,0.693111,0.693253,1437.000000


In [7]:
from aif360.sklearn import metrics as mt
from aif360.datasets import BinaryLabelDataset

pip install 'aif360[LawSchoolGPA]'


Create Binary Label Dataset out of training data

In [8]:
train_df = pd.merge(X_train,y_train,left_index=True,right_index=True)
train_df.drop('id',axis=1)

,sex,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,c_charge_desc,time_in_custody_in_days,time_in_jail_in_days,African-American_race,...,Other_race,Felony_c_charge_degree,Misdemeanor_c_charge_degree,High_score_text,Low_score_text,Medium_score_text,High_v_score_text,Low_v_score_text,Medium_v_score_text,two_year_recid
5375,1,20,0,0,0,0,0.592456,61.0,1.0,1,...,0,1,0,0,0,1,0,0,1,1
3605,1,43,0,0,0,2,0.383073,0.0,0.0,0,...,1,0,1,0,1,0,0,1,0,0
29,1,26,0,0,1,5,0.497714,8.0,0.0,1,...,0,1,0,0,0,1,0,0,1,1
2420,1,41,0,0,0,8,0.534040,0.0,1.0,1,...,0,1,0,0,1,0,0,1,0,1
5641,1,24,0,0,0,5,0.441770,39.0,37.0,0,...,0,1,0,0,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185,1,24,0,0,0,5,0.365278,31.0,1.0,0,...,1,1,0,0,0,1,0,0,1,0
6569,1,27,0,0,0,0,0.478449,1.0,12.0,0,...,0,1,0,0,1,0,0,1,0,0
4187,1,24,0,0,0,2,0.406297,9.0,49.0,1,...,0,0,1,1,0,0,0,0,1,1
4854,1,58,0,0,0,1,0.365514,61.0,2.0,0,...,0,0,1,0,1,0,0,1,0,0


In [25]:
train_df.columns

Index(['id', 'sex', 'age', 'juv_fel_count', 'juv_misd_count',
       'juv_other_count', 'priors_count', 'c_charge_desc',
       'time_in_custody_in_days', 'time_in_jail_in_days',
       'African-American_race', 'Asian_race', 'Caucasian_race',
       'Hispanic_race', 'Native American_race', 'Other_race',
       'Felony_c_charge_degree', 'Misdemeanor_c_charge_degree',
       'High_score_text', 'Low_score_text', 'Medium_score_text',
       'High_v_score_text', 'Low_v_score_text', 'Medium_v_score_text',
       'two_year_recid'],
      dtype='object')

In [26]:
favorable_label = 0
unfavorable_label = 1

privileged_groups = [{'sex': 1, 'African-American_race':0}]
unprivileged_groups = [{'sex': 0, 'African-American_race':1 }]


protected_attribute_names = ['sex', 'African-American_race']

In [27]:
train_df_blb = BinaryLabelDataset(favorable_label=favorable_label, unfavorable_label=unfavorable_label, df=train_df,label_names= ['two_year_recid'], protected_attribute_names = protected_attribute_names)

*explain why you dont want supression

*explain why not doing resampling

1. Applying reweighting

The reweighing algorithm works by assigning higher weights to examples from the underprivileged group and lower weights to examples from the privileged group. This helps to ensure that the model is not biased towards the privileged group.

In [11]:
from aif360.algorithms.preprocessing import Reweighing

Reweighting the training data

In [28]:
reweigher = Reweighing(unprivileged_groups = unprivileged_groups, privileged_groups= privileged_groups)
reweigher.fit(train_df_blb)
train_df_transformed = reweigher.transform(train_df_blb)

<b>Building the model on Reweighted data

In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import holisticai

<u>Scaling

In [29]:
# Create a StandardScaler object
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(train_df_transformed.features)
y_train = train_df_transformed.labels.ravel()

In [30]:
sample_weights = train_df_transformed.instance_weights.ravel()

train_df_transformed.instance_weights

<u> building the model

In [31]:
lmod = LogisticRegression()
lmod.fit(X_train_scaled, y_train,
        sample_weight=train_df_transformed.instance_weights.ravel())

LogisticRegression()

In [32]:
X_test_pred = X_test.copy()
X_test_scaled = scaler.transform(X_test_pred)
X_test_pred['y_pred_proba'] = lmod.predict_proba(X_test_scaled)[:, 0].reshape(-1,1)
X_test_pred['y_pred_label'] = np.where(X_test_pred['y_pred_proba'] > 0.5,1,0)


c:\Users\athar\anaconda3\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [33]:
from sklearn import metrics
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric

In [34]:
reweighting_perf = pd.DataFrame(classification_report(y_true=y_test, y_pred=X_test_pred['y_pred_label'],output_dict=True)).T
reweighting_perf

,precision,recall,f1-score,support
0,0.330658,0.260759,0.291578,790.00000
1,0.282555,0.355487,0.314853,647.00000
accuracy,0.303410,0.303410,0.303410,0.30341
macro avg,0.306607,0.308123,0.303216,1437.00000
weighted avg,0.309000,0.303410,0.302057,1437.00000


Recall reduced alot

Now let's check fpr

In [20]:
from holisticai.bias.metrics import false_positive_rate_diff

In [21]:
np.where(X_test_pred['sex'] == 0,1,0)

array([0, 0, 1, ..., 0, 1, 0])

In [35]:
group_males = np.where(X_test_pred['sex'] == 0,1,0)
group_females = np.where(X_test_pred['sex'] == 0,0,1)

group_african_american = np.where(X_test_pred['African-American_race'] == 1,1,0)
group_non_african_american = np.where(X_test_pred['African-American_race'] == 0,0,1)

In [36]:
y_pred_rwt = X_test_pred['y_pred_label']
y_true = y_test

In [37]:
false_positive_rate_diff(group_males,group_females,y_pred_rwt,y_true)

0.1685883079418623

Significantly came down to almost zero